# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from math import *
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Tweets,Relevância:
0,@immandy4 o arquiteto do ps5 disse que ele ser...,1
1,o vídeo que eu fiz pro evento tá no youtube tb...,0
2,@chief117_com_br @vingadorbrambz enquanto o da...,0
3,possivelmente hoje sai o valor e a data de lan...,1
4,"@lealemy_ sim, mas os produtos da sony vão par...",1


In [4]:
#test = pd.read_excel(filename, sheet_name = 'Teste')
#test.head(5)


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#Limpando caracteres:
def limpa_tweets(dataframe):
    train_limpo = dataframe.iloc[:, 0]
    ls= []
    for w in train_limpo:
        w = w.casefold()
        w = w.replace(":",'')
        w = w.replace('"','')
        w = w.replace("'",'')
        w = w.replace("(",'')
        w = w.replace(")",'')
        w = w.replace('!','')
        w = w.replace('...','')
        w = w.replace(',','')
        w = w.replace('\n','')
        w = w.replace('/','')
        w = w.replace(' a ',' ')
        w = w.replace(' e ',' ')
        w = w.replace(' o ',' ')
        w = w.replace(' de ',' ')
        w = w.replace(' do ',' ')
        w = w.replace(' no ',' ')
        w = w.replace(' na ',' ')
        w = w.replace(' os ',' ')
        w = w.replace(' que ',' ')
        w = w.replace(' q ',' ')
        w = w.replace(' as ',' ')
        w = w.replace(' - ',' ')
        w = ''.join(' ' + char+' ' if char in UNICODE_EMOJI else char for char in w)
        w = w.replace('  ',' ')
        w = w.strip()
        ls.append(w)
    dataframe['Tweets'] = ls
    return dataframe
train = limpa_tweets(train)


In [6]:
#train.to_excel(r'PS5Emoji.xlsx', sheet_name='Treinamento', index = False)

## Classificador:
Precisamos de:
P(R|T) = P(T|R) * P(R)/ P(T)




In [7]:
#Calculando Probabilidades:
def treinamento(dataframe):
    Rele = pd.Series(dtype=object)
    Irrele = pd.Series(dtype=object)
    i = 0
    for s in dataframe['Tweets']:
        lsWords = pd.Series(s.split())
        if dataframe['Relevância:'][i] == 1:
            Rele = Rele.append(lsWords)
        else:
            Irrele = Irrele.append(lsWords)
        i += 1
    return [Rele, Irrele]
    



In [8]:
Rele = treinamento(train)[0]
Irrele = treinamento(train)[1]
total_portugues = 50000 #Consideramos todas as palavras do portugues comumente usadas no twitter para maximizar smoothing.
#Calcaulando probabilidade em tweets relevantes:
lsRele = []
for v in Rele.value_counts():
    lsRele.append(abs(log10((v+1)/(total_portugues+Rele.count()))))
lsKeysRele = Rele.value_counts().index.tolist()
dic_rele = {lsKeysRele[i]: lsRele[i] for i in range(len(lsKeysRele))} 
print(dic_rele)






{'ps5': 2.6398485522173623, 'é': 3.12057393120585, 'um': 3.166331421766525, 'vai': 3.166331421766525, 'com': 3.1785658781835364, 'sony': 3.1785658781835364, 'hoje': 3.2041199826559246, 'preço': 3.2312722286995395, 'da': 3.2312722286995395, 'mais': 3.24551266781415, 'evento': 3.2912701583748247, 'em': 3.3246939138617746, 'não': 3.3246939138617746, 'pra': 3.3609060865162195, 'eu': 3.3609060865162195, 'se': 3.400414627799893, 'uma': 3.4438803215809832, 'dia': 3.4438803215809832, 'brasil': 3.4921850011555384, 'para': 3.5185139398778875, 'sobre': 3.5185139398778875, 'será': 3.5185139398778875, 'por': 3.5185139398778875, 'tem': 3.5185139398778875, 'ser': 3.546542663478131, 'pro': 3.546542663478131, 'lançamento': 3.5765058868555744, 'xbox': 3.5765058868555744, '#ps5': 3.5765058868555744, 'vão': 3.6434526764861874, '👍': 3.6434526764861874, 'of': 3.6434526764861874, 'só': 3.6434526764861874, 'comprar': 3.6434526764861874, 'series': 3.6434526764861874, 'já': 3.6434526764861874, 'ps5.': 3.6434526

In [9]:
#Calculando Probabilidade em tweets irrelevantes:
lsIrrele = []
for v in Irrele.value_counts():
    lsIrrele.append(abs(log10((v+1)/(total_portugues+Irrele.count()))))
lsKeysIrrele = Irrele.value_counts().index.tolist()
dic_irrele = {lsKeysIrrele[i]: lsIrrele[i] for i in range(len(lsKeysIrrele))}
print(dic_irrele)

{'ps5': 2.7094593305808745, 'um': 2.9741085282798463, 'é': 2.981933865791803, 'dia': 3.050198697350329, 'bom': 3.1540948312190515, 'pra': 3.1908176382437916, 'não': 3.2171465769661407, 'eu': 3.245175300566384, '#ps5': 3.245175300566384, 'hoje': 3.245175300566384, 'vai': 3.2598985573870904, 'da': 3.3420853135744406, 'me': 3.360568719268454, 'evento': 3.360568719268454, 'com': 3.360568719268454, 'tem': 3.4000772605521274, 'uma': 3.4000772605521274, 'para': 3.4435429543332177, 'mais': 3.4670240501827405, 'se': 3.491847633907773, 'xbox': 3.546205296230365, 'isso': 3.60835320297921, 'sony': 3.60835320297921, 'meu': 3.60835320297921, 'já': 3.6809038701278216, 'por': 3.6809038701278216, 'mas': 3.6809038701278216, 'preço': 3.6809038701278216, 'tá': 3.6809038701278216, 'ter': 3.6809038701278216, 'ela': 3.6809038701278216, 'dar': 3.7222965552860465, 'em': 3.7222965552860465, 'minha': 3.7222965552860465, 'aqui': 3.7680540458467218, '#ps5showcase': 3.7680540458467218, 'porra': 3.7680540458467218, 

In [10]:
def classifica(dataframe, dic_rele, dic_irrele):
    lsClassificacao = []
    for t in dataframe['Tweets']:
        lsWord = t.split()
        lsRele = []
        lsIrrele = []
        for w in lsWord:
            if w in dic_rele:
                lsRele.append(dic_rele[w])
            if w in dic_irrele:
                lsIrrele.append(dic_irrele[w])
            else:
                lsRele.append(1/(total_portugues+Rele.count()))
                lsIrrele.append(1/(total_portugues+Irrele.count()))
        if np.sum(lsRele) >= np.sum(lsIrrele):
            lsClassificacao.append(1)
        else:
            lsClassificacao.append(0)
    return lsClassificacao
teste = pd.Series(classifica(train, dic_rele, dic_irrele))
teste.value_counts()
train['Classificação'] = teste
pd.set_option("display.max_rows", None, "display.max_columns", None)
train

,Tweets,Relevância:,Classificação
0,@immandy4 arquiteto ps5 disse ele será muito s...,1,1
1,o vídeo eu fiz pro evento tá youtube tbm#ps5 #...,0,0
2,@chief117_com_br @vingadorbrambz enquanto data...,0,0
3,possivelmente hoje sai valor data lançamento p...,1,1
4,@lealemy_ sim mas produtos da sony vão parar s...,1,1
5,não oficial este modelo preto para #ps5 seria ...,1,1
6,@marcelo_gj1 @omattmeyer @lv_ogandojr @marccon...,1,1
7,hoje 1700 finalmente saberemos data preço ps5 ...,1,1
8,falta pouco hein hj às 17h promete #playstatio...,0,0
9,só falta ps5 agr 😭 httpst.cobwgw1hee77,0,0


In [11]:
teste.value_counts()

1    150
0    150
dtype: int64

In [12]:
#Comparando com as entradas manualmente digitadas:
train['Relevância:'].value_counts()

0    151
1    149
Name: Relevância:, dtype: int64

In [13]:
pd.crosstab(train['Relevância:'], train['Classificação'], normalize='index')

Classificação,0,1
Relevância:,,
0,0.986755,0.013245
1,0.006711,0.993289


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**